# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 04**: Data Unions & Joins Pipeline

**Date**: September 23rd 2025

**Student Name**: Luis Adrian Bravo Ramirez

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://4f3c6067fdf1:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/23 22:35:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!pwd

/opt/spark/work-dir


# Creating Agencies Dataframe

In [3]:
from pyspark.sql.functions import get_json_object 
from luisbravor00.spark_utils import SparkUtils

schema_agencies = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
df_agencies = spark.read.schema(schema_agencies).option("header", True).csv("/opt/spark/work-dir/data/car_service/agencies")
df_agencies = df_agencies.withColumn("agency_name", get_json_object(df_agencies.agency_info, '$.agency_name'))

# Creating Brands Dataframe

In [4]:
schema_brands = SparkUtils.generate_schema([("brand_id", "int"), ("brand_info", "string")])
df_brands = spark.read.schema(schema_brands).option("header", True).csv("/opt/spark/work-dir/data/car_service/brands")

# Creating Cars Dataframe

In [5]:
schema_cars = SparkUtils.generate_schema([("car_id", "int"), ("car_info", "string")])
df_cars = spark.read.schema(schema_cars).option("header", True).csv("/opt/spark/work-dir/data/car_service/cars")
df_cars = df_cars.withColumn("car_name", get_json_object(df_cars.car_info, '$.car_name'))

# Creating Customers Dataframe

In [6]:
schema_customers = SparkUtils.generate_schema([("customer_id", "int"), ("customer_info", "string")])
df_customers = spark.read.schema(schema_customers).option("header", True).csv("/opt/spark/work-dir/data/car_service/customers")
df_customers = df_customers.withColumn("customer_name", get_json_object(df_customers.customer_info, '$.customer_name'))

# Creating Rentals Dataframe

In [7]:
schema_rentals = SparkUtils.generate_schema([("rental_id", "int"), ("rental_info", "string")])
df_rentals = spark.read.schema(schema_rentals).option("header", True).csv("/opt/spark/work-dir/data/car_service/rentals")
df_rentals = df_rentals \
    .withColumn("car_id", get_json_object(df_rentals.rental_info, '$.car_id')) \
    .withColumn("customer_id", get_json_object(df_rentals.rental_info, '$.customer_id')) \
    .withColumn("agency_id", get_json_object(df_rentals.rental_info, '$.agency_id'))

# Obtaining final Dataframe


In [8]:
# Join all the dataframes
df_final = df_rentals \
    .join(df_cars, on='car_id', how='left') \
    .join(df_agencies, on='agency_id', how='left') \
    .join(df_customers, on='customer_id', how='left') \

# Filter out the values (not needed)

# Select the required columns
df_final = df_final.select("rental_id", "car_name", "agency_name", "customer_name")

df_final.show(truncate=False)

+---------+-----------------------------------+-------------+---------------+
|rental_id|car_name                           |agency_name  |customer_name  |
+---------+-----------------------------------+-------------+---------------+
|11891    |Wallace-Carlson Model 9            |NYC Rentals  |Margaret Jones |
|11892    |Grimes-Green Model 8               |LA Car Rental|Albert Williams|
|11893    |Stewart-Allen Model 5              |SF Cars      |Caleb Fleming  |
|11894    |Campos PLC Model 4                 |NYC Rentals  |Andrew Butler  |
|11895    |Wagner LLC Model 1                 |SF Cars      |Kristin Potts  |
|11896    |Jones, Jefferson and Rivera Model 7|LA Car Rental|Jeremy Parks   |
|11897    |Lopez and Sons Model 9             |Zapopan Auto |Terry Wells    |
|11898    |Salazar Ltd Model 8                |SF Cars      |Marc Williams  |
|11899    |Villanueva PLC Model 7             |LA Car Rental|Danny Williams |
|11900    |Faulkner-Howard Model 5            |SF Cars      |Eri

In [9]:
sc.stop()